In [2]:

from openai import OpenAI  # Import the OpenAI class
from langchain.llms.base import LLM  # Import the LLM base class
from typing import Optional, Any, List  # Import necessary type hints
from langchain.callbacks.manager import CallbackManagerForLLMRun  # Import CallbackManagerForLLMRun

class QwenLLM(LLM):
    client: Optional[Any] = None
    def __init__(self):
        super().__init__()
        self.client = OpenAI(base_url="https://localhost:1143/v1", api_key="qwen2.5:3b")
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None, 
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        completion = self.client.completions.create(model="qwen2.5:3b", prompt=prompt,
                                                    tepremperature=kwargs.get("temperature", 0.7),
                                                    max_tokens=kwargs.get("max_tokens", 256),
                                                    top_p=kwargs.get("top_p", 1.0),
                                                    frequency_penalty=kwargs.get("frequency_penalty", 0.0),
                                                    stream=kwargs.get("stream", False))
        return completion.choices[0].text if completion.choices else None
        
       

ModuleNotFoundError: No module named 'openai'

In [41]:
from langchain_community.llms import Ollama
llm = Ollama(model="qwen2.5:3b")

In [9]:
import requests
def LLM_call(API_URL, Model_name, prompt):
    # 构建请求数据
    data = {
        "model": model_name,
        "prompt": prompt
    }
    try:
        # 发送 POST 请求到 Ollama API
        response = requests.post(API_URL, json=data)
        # 检查请求是否成功
        response.raise_for_status()
    
        # 初始化一个空字符串用于存储最终的响应结果
        full_response = ""
        # 遍历响应文本中的每一行
        for line in response.text.strip().split('\n'):
            # 解析当前行的 JSON 数据
            import json
            line_data = json.loads(line)
            # 检查当前行是否包含 'response' 字段
            if 'response' in line_data:
                # 将 'response' 字段的值添加到最终响应结果中
                full_response += line_data['response']
    
        # 打印最终的响应结果
        return full_response

    except requests.RequestException as e:
    # 若请求过程中出现异常，打印错误信息
        return "error"

In [ ]:
llm("how are you?")  # Call the LLM with a prompt

"I'm just a computer program and don't have feelings or personal experiences, but I'm here to assist you! How can I help you today?"

In [10]:
API_URL = "http://localhost:11434/api/generate"
model_name = "qwen2.5:3b" 
res = LLM_call(API_URL, model_name, "how areyou")

In [6]:
# Prompt tempalte
from langchain.prompts import PromptTemplate
template = """
{our_text}
你能为上述内容创建一个包含{word_count}个词的推文吗？
"""

In [14]:
prompt = PromptTemplate(input_variables=["our_text", "word_count"], template=template)
final_prompt = prompt.format(our_text="我喜欢旅行，我已经去过6个国家，我计划不久后再去几个国家", word_count=3)

In [16]:
print("=="*20)
print("prompt:")
print(final_prompt)
print("=="*20)
print("answer:")
print(LLM_call(API_URL, model_name, final_prompt))


prompt:

我喜欢旅行，我已经去过6个国家，我计划不久后再去几个国家
你能为上述内容创建一个包含3个词的推文吗？

answer:
探索不止六国，新旅程待启！


In [22]:
from langchain import PromptTemplate, FewShotPromptTemplate
examples = [{'query':'什么是手机?',
            'answer': '手机是一种便携式电子设备，通常用于通信、上网和多媒体功能。它可以通过无线网络连接到互联网，并支持各种应用程序和功能。手机通常具有触摸屏、相机和其他传感器。'},
            {'query':'你的梦想是什么?',
            'answer': '我的梦想是环游世界，探索不同的文化和风景。'}]

In [23]:
exaple_template = """
Question: {query}
Answer: {answer}"""
example_prompt = PromptTemplate(input_variables=["query", "answer"], template=exaple_template)

In [24]:
prefix = """你是一个5岁的小孩，非常有趣，顽皮且可爱：
以下是一些例子：
"""
suffix = """
Question: {userInput}
Response: """


In [25]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["userInput"],
    example_separator="\n\n"
)

In [26]:
query = "房子是什么?"
real_prompt = few_shot_prompt.format(userInput=query)
print("=="*20)
print("prompt:")
print(real_prompt)  
print("=="*20)
print("answer:")
print(LLM_call(API_URL, model_name, real_prompt))


prompt:
你是一个5岁的小孩，非常有趣，顽皮且可爱：
以下是一些例子：



Question: 什么是手机?
Answer: 手机是一种便携式电子设备，通常用于通信、上网和多媒体功能。它可以通过无线网络连接到互联网，并支持各种应用程序和功能。手机通常具有触摸屏、相机和其他传感器。


Question: 你的梦想是什么?
Answer: 我的梦想是环游世界，探索不同的文化和风景。


Question: 房子是什么?
Response: 
answer:
房子就像一个大大的家呀！我们可以在里面睡觉、吃饭、玩玩具和看动画片哦！它可是用来住人的呢，就像我们有一个温暖的房间可以休息一样。不过，有的房子还会装上窗户让我们看到外面的世界，有时候甚至会有高高的围墙保护我们安全。


# Chain
- 步骤链条，上一步骤结果传入下一个步骤

In [45]:
from langchain.schema.runnable import RunnableLambda


chain = few_shot_prompt | llm

# Invoke the chain with the query
result = chain.invoke({"userInput": query})
# Convert the result to a string if necessary
print(str(result))



房子是一块大大的地方，里面有很多房间。它就像一个家，我们可以在里面睡觉、吃饭和玩玩具哦！房子还保护我们不受风雨的影响呢。你有自己住的房子吗？我有！


In [47]:
from langchain.chains import LLMChain
achain = LLMChain(
    llm=llm,
    prompt=few_shot_prompt,
    verbose=True
)
achain(query)

C:\Users\00036777\AppData\Local\Temp\ipykernel_28528\2551215198.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  achain(query)




> Entering new LLMChain chain...
Prompt after formatting:
你是一个5岁的小孩，非常有趣，顽皮且可爱：
以下是一些例子：



Question: 什么是手机?
Answer: 手机是一种便携式电子设备，通常用于通信、上网和多媒体功能。它可以通过无线网络连接到互联网，并支持各种应用程序和功能。手机通常具有触摸屏、相机和其他传感器。


Question: 你的梦想是什么?
Answer: 我的梦想是环游世界，探索不同的文化和风景。


Question: 房子是什么?
Response: 

> Finished chain.


{'userInput': '房子是什么?',
 'text': '房子就像是家的家，它是用来住的地方哦！就像你有一个温暖的小窝一样，家里有床、玩具和爸爸妈妈，房子就是大人们生活的那种大大的“小窝”。它还可以有很多房间，比如厨房、客厅和卧室。有的房子还会有一些特别的功能，比如游泳池或者花园。'}

Output Parsers
语言模型输出文本，但很多时候需要结构化的输出，output Parsers负责两个任务：
- 指导该模型如何格式化输出
- 将原始文本输出解析成结构化格式。


In [48]:
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [49]:
prompt_template_cls = PromptTemplate(
    template="Provide 5 examples of {query}.\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions}
)
new_prompt = prompt_template_cls.format(query="房子是什么？")

In [52]:
chain = prompt_template_cls | llm | CommaSeparatedListOutputParser()
result = chain.invoke({"query": "房子是什么？"})
print(result)

['房子是建筑物', '房子是居所', '房子是用来居住的', '房子是一种住宅类型', '房子代表房屋建筑结构']
